<img src='https://github.com/wekeo/ai4EM_MOOC/raw/04147f290cfdcce341f819eab7ad037b95f25600/img/ai4eo_logos.jpg' alt='Logo EU Copernicus WEkEO' align='center' width='100%'></img>


# WEKEO TRAINING

<div style="text-align: right"><i> INTERMEDIATE LEVEL </i></div>

***
<center><h1>  Temporal evolution of permafrost </h1></center>

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Run" and select "Run All Above Selected Cell".<br>

**General Note 3**: To explore more (Python and R) content, there is our [**Jupyter Catalogue**](https://notebooks.apps.mercator.dpi.wekeo.eu/), and if you seek support, there are plenty of useful resources in our [**Help Center**](https://help.wekeo.eu/en/). Feel free to contact us using our [**live chat widget**](https://www.wekeo.eu/support) or asynchronously at [support@wekeo.eu](support@wekeo.eu) ! <br>

***

# Table of contents
- [1. Introduction](#1.-Introduction)
- [2. Setting up the R environment](#2.-Setting-up-the-R-environment)
- [3. Data Access](#3.-Data-Access)
- [4. Input](#4.-Input)
- [5. Plot](#5.-Plot)
    - [5.1. Basic Plot of skin temperature](#5.1.-Basic-Plot-of-skin-temperature)
    - [5.2. Visualization seasonnal variation of skin temperature](#5.2.-Visualization-seasonnal-variation-of-skin-temperature)
    - [5.3. Visualization of soil temperature at a single point](#5.3.-Visualization-of-soil-temperature-at-a-single-point)
    - [5.4. Visualization of soil temperature at specific month](#5.4.-Visualization-of-soil-temperature-at-specific-month)
    - [5.5. Comparison between temperature and CH4](#5.5.-Comparison-between-temperature-and-CH4)
- [6. Conclusion](#6.-Conclusion)


# 1. Introduction

[Go back to the "Table of contents"](#Table-of-contents)

This training focuses on the early thaw of permafrost and its impact on methane emissions in Alaska. In recent years, Arctic regions have experienced unprecedented warming, with ground temperatures rising well above historical averages, leading to the premature thawing of permafrost layers.

Permafrost soils store vast amounts of organic matter and methane that have remained frozen for thousands of years. As the permafrost thaws, anaerobic microbial activity increases, releasing methane (CH₄) into the atmosphere a potent greenhouse gas more than 25 times stronger than CO2. In some regions, particularly in areas of waterlogged tundra and peatlands, this process has led to significant increases in atmospheric methane concentrations.

This training will allow you to explore the spatial and temporal evolution of methane concentrations over Alaska using reanalysis data. You will visualize trends with time series and maps of temperature at 2m, soil temperature and methane with ERA5 and CAMS data. Time series will illustrate the trends in annual mean temperature and methane concentration, accompanied by linear regression lines.

# 2. Setting up the R environment
[Go back to the "Table of contents"](#Table-of-contents)

The Jupyter Notebook must be set up with all the necessary available tools from the Jupyter Notebook ecosystem. Here is the list of the modules we will be using in this exercise.

| Module name | Description |
| :---: | :---|
| **ggplot2** |[ggplot2](https://ggplot2.tidyverse.org/) is a system for declaratively creating graphics. |
| **dplyr** |[dplyr](https://dplyr.tidyverse.org/) is a library for manipulating data. |
| **RColorBrewer** |[RColorBrewer](https://cran.r-project.org/web/packages/RColorBrewer/index.html) is a library for color palettes in plot. |
| **ncdf4** |[ncdf4](https://cran.r-project.org/web/packages/ncdf4/index.html) is an interface for the 'NetCDF' file formats. |
| **gridExtra** |[gridExtra](https://cran.r-project.org/web/packages/gridExtra/index.html) is a library for arranging multiple grid-based plots on a page. |
| **scales** |[scales](https://cran.r-project.org/web/packages/scales/index.html) is a library for determing Graphical scales map data to aesthetics.
| **lubridate** |[lubridate](https://www.rdocumentation.org/packages/lubridate/versions/1.9.3) is a library for fast and user-friendly analysis of date-time data. |


In [ ]:
# Modules system
# for ignoring Warning message
options(warn = -1) 

## Load Packages
library(ggplot2)        # System to creating graphycs
library(dplyr)          # Operators for data manipulation
library(RColorBrewer)   # For color palettes
library(ncdf4)          # High-level interface to netCDF data files
library(gridExtra)      # To arrange multiple grid-based plots on a page
library(lubridate)      # Fast and user friendly parsing of date-time data
library(scales)        # Scales map data to aesthetics

If the required R libraries are not installed, you can use the following command to install them:

<center><h4> install.packages(c("ggplot2", "dplyr", "RColorBrewer", "ncdf4", "gridExtra", "lubridate","scales")) </h4></center>

# 3. Data Access

[Go back to the "Table of contents"](#Table-of-contents)


From the [WEKEO viewer](https://www.wekeo.eu/data), you can explore all the products available with many filters to select the region you are interested in, the parameters you want to study, etc.

WEkEO is a European platform providing access to environmental data, developed by the European Union in partnership with agencies like ESA (European Space Agency), EUMETSAT, and Mercator Ocean. It provides access to a wide range of Earth observation data.

For this training, we will use the following data:

* **Full Name**:CAMS_GLOBAL_REANALYSIS_EAC4
* **Spatial region**: Alaska area from Lat=(54ºN, 74ºN), Lon=(-168ºE,-146ºE)
* **Elevation**: At 1000 hPa (near the ground)
* **Variables**: methane
* **Time period**: monthly means from 2003-01-01 to 2020-12-01

 and: 
* **Full Name**:ERA5-Land monthly averaged data from 1950 to present
* **Spatial region**: Alaska area from Lat=(54ºN, 74ºN), Lon=(-168ºE,-146ºE)
* **Variables**: temperature at 2m, skin temperature, soil temperature
* **Time period**: monthly means from 2000-01-01 to 2023-12-01

**You don't have to download it**, the files are already downloading in this training.

__Optional:__ Go to the `Data store` and try to download this data selection. Note: You'll need to have your own WEKEO credentials -- username and password. Creating an account is free of charge and available [here](https://www.wekeo.eu/register).

# 4. Input
[Go back to the "Table of contents"](#Table-of-contents)


In [ ]:
# Set the path for the NetCDF files
file <- "data/data_era5.nc"

In [ ]:
# Load NetCDF File
data <- nc_open(file)

In [ ]:
# list of variables
names(data$var)

# list of dimensions
names(data$dim)

In [ ]:
# Access Data
latitude <- ncvar_get(data, "latitude")  # Name of the latitude variable
longitude <- ncvar_get(data, "longitude")  # Name of the longitude variable
time <- ncvar_get(data, "valid_time")  # Name of the time variable

skin_temperature <- ncvar_get(data, "skt")  -273.15   # skin_temperature
t2m <- ncvar_get(data, "t2m")  -273.15    # 2 metre temperature

In [ ]:
# visualisation of variable
print(dim(t2m)) # know the dimension of the variable
print (t2m[100,100,2]) # know the value depending on longitude, latitude and time

In [ ]:
# visualisation of variable time
print (time[1]) # know the value of the first time

In [ ]:
# have information about the unit of time variable
ncatt_get(data, 'valid_time')

In [ ]:
# Variable time is the number of seconds since 1970-01-01 00:00:00, we should convert:
time_values <- as.POSIXct(time, origin = "1970-01-01", tz = "GMT")
time_values[1:10]

In [ ]:
# to save memory
rm(data)

# 5. Plot

[Go back to the "Table of contents"](#Table-of-contents)
    
## 5.1. Basic Plot of skin temperature
[Go back to the "Table of contents"](#Table-of-contents)

In [ ]:
# we create a dataframe
index_time=1
df <- data.frame(expand.grid(longitude, latitude), skin_temperature = as.vector(skin_temperature[,,index_time]))
names(df) <- c("Longitude", "Latitude", "skin_temperature")

In [ ]:
# visualisation of the dataframe
df[1:10,]

In [ ]:
# Plotting
p <- ggplot() +

# Add skin temperature data 
geom_tile(data = df, aes(x=Longitude, y=Latitude, fill = skin_temperature)) +

# Color scale options limits = c()
scale_fill_gradientn(colors = brewer.pal(9, "OrRd"),  name="Temperature [°C]",limits = c(), na.value = "grey") +


# General aesthetic options
theme_light() +

# Title and subtitle
labs(title = paste("Skin temperature in °C - ",time_values[index_time],sep=''),
     x = "Longitude",
     y = "Latitude")+

# Legend position and aesthetic options
theme_minimal()+

theme(  axis.text = element_blank(), 
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20),  # Increase the size of the y-axis title
    legend.text = element_text(size = 20),    # Increase the size of the legend text
    legend.title = element_text(size = 20)     # Increase the size of the legend title
)   +

borders("world", colour = "gray80") + # add country borders
coord_sf(crs = 3857, xlim = c(-165, -145), ylim = c(55, 75)) # applies the Web Mercator projection 

# to save the figure
ggsave("./figures/basic_figure.png", plot = p, width = 12, height = 6, dpi = 300)

IRdisplay::display_png(file = "./figures/basic_figure.png", width = 1000, height = 800)

This map shows the skin temperature (in degrees) in Alaska in January 2020. Skin temperature is higher in the southern part of the state.

## 5.2. Visualization seasonnal variation of skin temperature
[Go back to the "Table of contents"](#Table-of-contents)
    
Now that we know how to visualize a map of skin temperature, we will examine the evolution of the skin temperature over the course of a year. To do this, we will compute the monthly mean fields from 2000 to 2023, and then plot all the corresponding maps of: january, april, july and october.

Here are the main steps we will follow to analyze the evolution of temperature over a typical year:
- Create a dataframe of the temperature along with spatial and temporal coordinates.
- Compute the monthly mean temperature from 2000 to 2023 by averaging all data for each calendar month across the time range.
- Plot skin temperature maps for each january, april, july and october mean.

In [ ]:
# creating a dataframe of current variable 
df <- data.frame(expand.grid(longitude, latitude,time_values), skin_temperature = as.vector(skin_temperature))
names(df) <- c("Longitude", "Latitude","time", "skin_temperature")

# add the corresponding month
df$month <- as.numeric(format(df$time, "%m"))

# dataframe visualization
df[1:10,]

In [ ]:
# average over each month 
monthly_means <- df %>% 
    group_by(Longitude, Latitude, month) %>%
    summarise(mean_temperature = mean(skin_temperature,na.rm=FALSE))

# and we convert the results in dataframe. Because after the operation the dataframe became an object of tibble type 
monthly_means<- as.data.frame(monthly_means)

In [ ]:
# création of plot function
mapping_dataframe <- function(df,var_name,date,ind){
    options(repr.plot.width = 8, repr.plot.height = 6) 

    p <- ggplot(df, aes(x = Longitude, y = Latitude)) +

    # Add skin temperature data 
    geom_tile(aes(fill = mean_temperature)) +
    
    # Color scale options limits
    scale_fill_gradient2(low = "blue",mid='white', high = "red",midpoint=0,limits = c(-30,20), name = "Temperature (°C)", oob = squish) +

    # Title and subtitle
    labs(title = paste(" Skin temperature [°C] - ",date,sep=''),
         x = "Longitude",
         y = "Latitude")+

    # Legend position and aesthetic options
    theme_minimal()+
    theme_light() +

    theme(
        axis.text = element_blank(),
        plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
        axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
        axis.title.y = element_text(size = 20),  # Increase the size of the y-axis title
        legend.text = element_text(size = 20),    # Increase the size of the legend text
        legend.title = element_text(size = 20)     # Increase the size of the legend title
    )   +
    guides(fill = guide_colorbar(barwidth = 2, barheight = 15)) + # Adjust the size of the colorbar
    borders("world", colour = "gray80") + # add country borders
    coord_sf(crs = 3857, xlim = c(-165, -145), ylim = c(55, 75)) # applies the Web Mercator projection
    
    # to save the figure
    ggsave(paste("./figures/Skin_temperature_", date,".png",sep=""), plot = p, width = 12, height = 12, dpi = 300)
}

We load the creation of each figure:

In [ ]:
# creation of plot
month<- c('January','April','July','October')
mapping_dataframe (subset(monthly_means,month==1),'mean_temperature',month[1])
mapping_dataframe (subset(monthly_means,month==4),'mean_temperature',month[2])
mapping_dataframe (subset(monthly_means,month==7),'mean_temperature',month[3])
mapping_dataframe (subset(monthly_means,month==10),'mean_temperature',month[4])

In [ ]:
# Display January
IRdisplay::display_png(file = "./figures/Skin_temperature_January.png", width = 600, height = 400)

# Display April
IRdisplay::display_png(file = "./figures/Skin_temperature_April.png", width = 600, height = 400)

# Display July
IRdisplay::display_png(file = "./figures/Skin_temperature_July.png", width = 600, height = 400)

# Display October
IRdisplay::display_png(file = "./figures/Skin_temperature_October.png", width = 600, height = 400)

There is a strong seasonal variability, with temperature differences exceeding 40°C in some areas between winter and summer. During winter, most of Alaska is covered with snow and ice, and the ground temperature remains below 0°C.

## 5.3. Visualization of soil temperature at a single point
[Go back to the "Table of contents"](#Table-of-contents)


In the previous figure, we saw a high variation of skin temperature between winter and summer especially in the north of Alaska. We are looking the variation at a single point with the following coordinates: 
- Longitude : -156.8°E
- Latitude: 71.3°N

We will visualize the soil temperature at this point over a long period: January 2000 to December 2023.



In [ ]:
file <- "data/data_era5_stl.nc"
data <- nc_open(file)

# Access Data
latitude <- ncvar_get(data, "latitude")  # Name of the latitude variable
longitude <- ncvar_get(data, "longitude")  # Name of the longitude variable
time <- ncvar_get(data, "valid_time")  # Name of the time variable
time_values <- as.POSIXct(time, origin = "1970-01-01", tz = "GMT")

stl1 <- ncvar_get(data, "stl1") - 273.15    # soil temperature level 1
stl2 <- ncvar_get(data, "stl2") - 273.15    # soil temperature level 2
stl3 <- ncvar_get(data, "stl3") - 273.15    # soil temperature level 3
stl4 <- ncvar_get(data, "stl4") - 273.15    # soil temperature level 4
t2m <- ncvar_get(data, "t2m")  -273.15    # 2 metre temperature

# to save memory
rm(nc1)

We use the line below to define the index of longitude and latitude that we should use.

In [ ]:
# find index of the neareast coordinates
index_longitude <- which.min(abs(longitude-(-156.8)))
index_latitude <- which.min(abs(latitude-71.3))

# print the closest latitude and longitude
print(longitude[index_longitude])
print(latitude[index_latitude])

In [ ]:
# find index of time
index1 <- which.min(abs(difftime(time_values, ymd_hms("2000-01-01 00:00:00"), units = "secs")))
index2 <- which.min(abs(difftime(time_values, ymd_hms("2023-12-31 23:00:00"), units = "secs")))

Now we create a  dataframe for a given longitude and latitude. This way, the dataframe will only be dependent of time.

In [ ]:
# create a dataframe 
df_stl=data.frame(date=time_values,
                  t2m=t2m[index_longitude,index_latitude,],
                 stl1=stl1[index_longitude,index_latitude,],
                 stl2=stl2[index_longitude,index_latitude,],
                 stl3=stl3[index_longitude,index_latitude,],
                 stl4=stl4[index_longitude,index_latitude,])[index1:index2,]

# dataframe visualization
df_stl[1:10,]

In [ ]:
# Determine figure size
options(repr.plot.width = 11, repr.plot.height = 6)

# Plot the graph with ggplot2
ggplot(df_stl, aes(x = date)) +
    geom_line(aes(y = stl1, color = "Level 1 [0-7cm]"), size = 1) +
    geom_line(aes(y = stl2, color = "Level 2 [7-28cm]"), size = 1) +
    geom_line(aes(y = stl3, color = "Level 3 [28-100cm]"), size = 1) +
    geom_line(aes(y = stl4, color = "Level 4 [100-289cm]"), size = 1) +

  ggtitle("Temperature of the soil at different level") +
  xlab("date") +
  ylab("Temperature [°C]") +
  scale_color_manual(
    name = "Parameters", 
    values = c("blue","red","green","black"))+
  theme_minimal() +

  theme(
    legend.position = c(0.91, 0.15),
    legend.title = element_text(size = 14),   # Increase tge size of legend title
    legend.text = element_text(size = 12),    # Increase tge size of text in the legend
    legend.key.size = unit(0.5, "cm"),  
    axis.text.x = element_text(size = 20),  # Increase the size of the x-axis values
    axis.text.y = element_text(size = 20),  # Increase the size of the y-axis values
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20)    # Increase the size of the y-axis title
      )


There is a significant variation in soil temperature across different depths. The deeper the level (e.g., level 4), the smaller the seasonal variations, generally ranging between -6°C and -2°C compared to the surface layer, which varies between -7°C and 7°C. Additionally, there is a noticeable time lag between upper and lower layers: the temperature maximum or minimum occurs earlier in the upper layers (e.g., level 1) than in the deeper ones (e.g., level 4). This is because atmospheric temperatures have a greater influence on the upper soil layers.

## 5.4. Visualization of soil temperature at specific month
[Go back to the "Table of contents"](#Table-of-contents)

In this section, we will produce the same type of plots as before, but focusing on annual variations for a specific month. To do this, we will filter the dataset to keep only the values corresponding to the selected month

In [ ]:
# for may
df_stl_month <- df_stl[format(df_stl$date, "%m") == "05", ]
df_stl_month

In [ ]:
# Determine figure size
options(repr.plot.width = 11, repr.plot.height = 6)

# Plot the graph with ggplot2
ggplot(df_stl_month, aes(x = date)) +
    geom_line(aes(y = stl1, color = "Level 1 [0-7cm]"), size = 1) +
    geom_line(aes(y = stl2, color = "Level 2 [7-28cm]"), size = 1) +
    geom_line(aes(y = stl3, color = "Level 3 [28-100cm]"), size = 1) +
    geom_line(aes(y = stl4, color = "Level 4 [100-289cm]"), size = 1) +

  ggtitle("Temperature of the soil at different level for May") +
  xlab("date") +
  ylab("Temperature [°C]") +
  scale_color_manual(
    name = "Parameters", 
    values = c("blue","red","green","black"))+
  theme_minimal() +

  theme(
    legend.position = c(0.9, 0.2),
    legend.title = element_text(size = 16),   # Increase tge size of legend title
    legend.text = element_text(size = 14),    # Increase tge size of text in the legend
    legend.key.size = unit(0.5, "cm"),  
    axis.text.x = element_text(size = 20),  # Increase the size of the x-axis values
    axis.text.y = element_text(size = 20),  # Increase the size of the y-axis values
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20)    # Increase the size of the y-axis title
      )

We observe a gradual increase in temperature during the month of May over the years, which indicates an earlier thawing of the soil. Let's now take a look at other months.

In [ ]:
# for October
df_stl_month <- df_stl[format(df_stl$date, "%m") == "10", ]

# Determine figure size
options(repr.plot.width = 11, repr.plot.height = 6)

# Plot the graph with ggplot2
ggplot(df_stl_month, aes(x = date)) +
    geom_line(aes(y = stl1, color = "Level 1 [0-7cm]"), size = 1) +
    geom_line(aes(y = stl2, color = "Level 2 [7-28cm]"), size = 1) +
    geom_line(aes(y = stl3, color = "Level 3 [28-100cm]"), size = 1) +
    geom_line(aes(y = stl4, color = "Level 4 [100-289cm]"), size = 1) +

  ggtitle("Temperature of the soil at different level for October") +
  xlab("date") +
  ylab("Temperature [°C]") +
  scale_color_manual(
    name = "Parameters", 
    values = c("blue","red","green","black"))+
  theme_minimal() +

  theme(
    legend.position = c(0.9, 0.2),
    legend.title = element_text(size = 16),   # Increase tge size of legend title
    legend.text = element_text(size = 14),    # Increase tge size of text in the legend
    legend.key.size = unit(0.5, "cm"),  
    axis.text.x = element_text(size = 20),  # Increase the size of the x-axis values
    axis.text.y = element_text(size = 20),  # Increase the size of the y-axis values
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20)    # Increase the size of the y-axis title
      )

In October, we observe a gradual increase in temperature over the years, resulting in a later refreezing of the soil

## 5.5. Comparison between temperature and CH4
[Go back to the "Table of contents"](#Table-of-contents)

In this section, we will compare the evolution of temperature with the evolution of methane. To do so, we will follow these steps:
- Open the methane dataset
- Create a dataframe for a single point
- Merge the temperature and methane dataframes
- Create annual mean
- Generate the corresponding plots

In [ ]:
file <- "data/data_cams.nc"
nc1 <- nc_open(file)

ch4 <- ncvar_get(nc1, "ch4")
time_ch4 <- ncvar_get(nc1, "valid_time")
longitude_ch4 <- ncvar_get(nc1, "longitude")
latitude_ch4 <- ncvar_get(nc1, "latitude")

time_ch4<- as.POSIXct(time_ch4, origin = "1970-01-01", tz = "GMT")

# to save memory
rm(nc1)

The coordinate that we want to study is the same as for CH4: 
- Longitude : -156.8°E
- Latitude: 71.3°N

We use the the line below to define the index of longitude and latitude that we should use.

In [ ]:
# find index of the neareast coordinates
index_longitude <- which.min(abs(longitude_ch4-(-156.8)))
index_latitude <- which.min(abs(latitude_ch4-71.3))

# print the closest latitude and longitude
print(longitude_ch4[index_longitude])
print(latitude_ch4[index_latitude])

In [ ]:
# find index of time
index1 <- which.min(abs(difftime(time_ch4, ymd_hms("2000-01-01 00:00:00"), units = "secs")))
index2 <- which.min(abs(difftime(time_ch4, ymd_hms("2019-12-01 23:00:00"), units = "secs")))

In [ ]:
# create a dataframe  of CH4
df_ch4=data.frame(date=time_ch4,ch4=ch4[index_longitude,index_latitude,])[index1:index2,]

# dataframe visualization
df_ch4[1:10,]

Finally, we merge the two dataframes: temperature and CH4 datasets.

In [ ]:
# Merge the two dataframes
df_final <-df_stl %>%
  left_join(df_ch4, by = "date")
df_final <- na.omit(df_final)
# dataframe visualization
df_final[1:10,]

In [ ]:
# annual mean
df <- df_final %>%
  mutate(year = year(date))

df_annual <- df %>%
  group_by(year) %>%
  summarise(
    t2m_mean = mean(t2m, na.rm = TRUE),
    ch4_mean = mean(ch4, na.rm = TRUE),
    stl1_mean = mean(stl1, na.rm = TRUE),
    stl2_mean = mean(stl2, na.rm = TRUE),
    stl3_mean = mean(stl3, na.rm = TRUE),
    stl4_mean = mean(stl4, na.rm = TRUE)
  )
df_annual

We create new columns to have normalized values:

In [ ]:
df_annual <- df_annual %>%
  mutate(
    t2m_norm =   (t2m_mean -  min(t2m_mean))  / (max(t2m_mean) - min(t2m_mean)),
    ch4_norm  =  (ch4_mean  - min(ch4_mean))  / (max(ch4_mean)  - min(ch4_mean)),
    stl1_norm  = (stl1_mean  - min(stl1_mean))  / (max(stl1_mean)  - min(stl1_mean)),
    stl2_norm  = (stl2_mean  - min(stl2_mean))  / (max(stl2_mean)  - min(stl2_mean)),
    stl3_norm  = (stl3_mean  - min(stl3_mean))  / (max(stl3_mean)  - min(stl3_mean)),
    stl4_norm  = (stl4_mean  - min(stl4_mean))  / (max(stl4_mean)  - min(stl4_mean))
  )
df_annual

In [ ]:
# Determine figure size
options(repr.plot.width =11, repr.plot.height = 6) 
# Plot the graph with ggplot2
ggplot(df_annual, aes(x = year)) +
    geom_line(aes(y = ch4_norm, color = "ch4"), size = 1) +
    geom_line(aes(y = t2m_norm, color = "t2m"), size = 1) +

    scale_y_continuous(name = "Normalized values", ) +
    ggtitle("Normalized values of mass of CH4 (kg/kg) and temperature at 2m for each year") +
    scale_color_manual(
    name = "Parameters", 
    values = c("blue", "red")) +
    theme_minimal() +

  theme(
    legend.position = c(0.2, 0.8),
    legend.title = element_text(size = 15),   # Increase tge size of legend title
    legend.text = element_text(size = 12),    # Increase tge size of text in the legend
    legend.key.size = unit(1, "cm"),  
    axis.text.x = element_text(size = 15),  # Increase the size of the x-axis values
    axis.text.y = element_text(size = 15),  # Increase the size of the y-axis values
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20)    # Increase the size of the y-axis title
      )


Both curves show an overall increasing trend in temperature and methane concentration over the years. This rise is particularly noticeable from 2012 , where an acceleration in the values can be observed. 

A linear regression can be applied to highlight and quantify the evolution of both trends:

In [ ]:
# Determine figure size
options(repr.plot.width =11, repr.plot.height = 6) 
# Plot the graph with ggplot2
ggplot(df_annual, aes(x = year)) +
    geom_line(aes(y = ch4_norm, color = "ch4"), size = 1) +
    geom_line(aes(y = t2m_norm, color = "t2m"), size = 1) +
  
# Temperature regression
  geom_smooth(aes(y = t2m_norm, color = "t2m"),
              method = "lm", linetype = "dashed", se = FALSE) +

  #  CH₄ regression
  geom_smooth(aes(y = ch4_norm, color = "ch4"),
              method = "lm", linetype = "dashed", se = FALSE) +
    scale_y_continuous(name = "Normalized values", ) +
    ggtitle("Normalized values of mass of CH4 (kg/kg) and temperature at 2m for each year") +
    scale_color_manual(
    name = "Parameters", 
    values = c("t2m" = "red", "ch4" = "blue")) +
    theme_minimal() +

  theme(
    legend.position = c(0.2, 0.8),
    legend.title = element_text(size = 15),   # Increase tge size of legend title
    legend.text = element_text(size = 12),    # Increase tge size of text in the legend
    legend.key.size = unit(1, "cm"),  
    axis.text.x = element_text(size = 15),  # Increase the size of the x-axis values
    axis.text.y = element_text(size = 15),  # Increase the size of the y-axis values
    plot.title = element_text(hjust = 0.5, size = 20),  # Center and increase the size of the title
    axis.title.x = element_text(size = 20),  # Increase the size of the x-axis title
    axis.title.y = element_text(size = 20)    # Increase the size of the y-axis title
      )


Here, we observe a steady increase in both temperature and methane concentration. In this example, the two parameters show similar trends over time. However, the rise in methane levels is not solely due to increasing temperatures and the thawing of permafrost (or vice versa). Other, more complex processes are also involved.

# 6. Conclusion
[Go back to the "Table of contents"](#Table-of-contents)

<div class="alert alert-block alert-success">
<b>Congratulations!</b> You have successfully completed the introductory-intermediate tutorial on using WEKEO products to evaluate the melting of snow, ice in ALaska. Throughout this tutorial, we have explained the basic tools necessary to access and visualize reanalysis models, generate different types of plots.
<br><br>

In this tutorial, you acquired all the information you need to:
 


* Access NetCDF datasets and Multispectral images.

* Navigate through the different variables, dimensions, and attributes of a NetCDF file.

* Plot maps of any variable.

* Modify maps to include additional information.

 
We sincerely hope that you have enjoyed the tutorial and found useful information in it. Please keep in mind that the tutorial has a progressive difficulty, moving quickly from basic elements to intermediate levels. Our intention is for all users to find useful information tailored to their level.
 
We understand that, for a user without prior knowledge, fully understanding all the procedures in the tutorial may be a challenge that requires some effort. However, we encourage everyone to take on the challenge as this is just the beginning of a journey towards a new understanding of the ocean and its ecosystems.
 
</div>

 